In [ ]:
# Add this cell after initial setup
import gc
import torch  # Import torch

def clear_memory():
    global pipe  # Declare pipe as global
    # Check if 'pipe' is defined in the global scope before accessing it
    if 'pipe' in globals() and pipe is not None:
        del pipe
    gc.collect()
    torch.cuda.empty_cache()

# Run this before switching models
clear_memory()

In [6]:
def on_generate(b):
    with output_display:
        output_display.clear_output()
        try:
            global pipe
            # Fixed indentation for model switching logic
            if pipe.__class__.__name__ != model_selector.value:
                pipe = load_model(model_selector.value)  # Properly indented block
                print(f"Loaded new model: {model_selector.value}")

            # Generation logic
            logo = generate_logo(
                "Tech company logo with circuit patterns",
                "low quality, 3D render"
            )
            display(logo)

            # Save to Colab storage
            logo.save("generated_logo.png")

        except Exception as e:
            print(f"Error: {str(e)}")
            torch.cuda.empty_cache()

In [7]:
# Define the condition variable before using it in the 'if' statement
condition = True  # Or False, depending on your desired logic

if condition:
    # All indented code (4 spaces)
    # Replace statement1 and statement2 with your actual code
    print("Condition is True")
# Back to normal indentation

Condition is True


In [ ]:
%%capture
!pip install diffusers accelerate transformers

### 1. Proper Model Configuration ###
MODEL_OPTIONS = [
    ("SSD-1B (Fast)", "Segmind/SSD-1B"),
    ("SDXL Base", "stabilityai/stable-diffusion-xl-base-1.0"),
    ("Playground v2.5", "playgroundai/playground-v2.5-1024px-aesthetic"),
    ("Analog Style", "wavymulder/Analog-Diffusion"),
    ("OpenDalle", "dataautogpt3/OpenDalleV1.1")
]

### 2. Revised Model Loading System ###
from diffusers import DiffusionPipeline
import torch

current_model_id = MODEL_OPTIONS[0][1]  # Initialize with first model
pipe = None

def load_model(model_id):
    global pipe
    if pipe is not None:
        del pipe
        torch.cuda.empty_cache()

    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16"
    ).to("cuda")
    return pipe

# Initial load
load_model(current_model_id)

### 3. Fixed Dropdown Implementation ###
from ipywidgets import Dropdown, Button, VBox, Output

model_selector = Dropdown(
    options=MODEL_OPTIONS,
    value=current_model_id,  # Match the initial value
    description="Model:"
)

generate_btn = Button(description="Generate Logo")
output_display = Output()

### 4. Error-Corrected Generation Handler ###
def on_generate(b):
    global current_model_id
    with output_display:
        output_display.clear_output()
        try:
            # Handle model switching
            if model_selector.value != current_model_id:
                load_model(model_selector.value)
                current_model_id = model_selector.value

            # Generate logo
            logo = pipe(
                prompt="Professional logo design, tech company, circuit patterns, vector art",
                negative_prompt="amateurish, blurry, text",
                num_inference_steps=25,
                guidance_scale=7.5
            ).images[0]

            display(logo)
            logo.save("logo.png")

        except Exception as e:
            print(f"Error: {str(e)}")
            torch.cuda.empty_cache()

generate_btn.on_click(on_generate)
display(VBox([model_selector, generate_btn, output_display]))

In [15]:
def load_model(model_id):
    global pipe
    if pipe is not None:
        del pipe
        torch.cuda.empty_cache()

    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16"
    ).to("cpu")  # Change to "cpu"
    return pipe

In [ ]:
import torch

    if torch.cuda.is_available():
        print("CUDA is available. Device count:", torch.cuda.device_count())
    else:
        print("CUDA is not available. Using CPU.")

In [17]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available. Device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Using CPU.")

CUDA is not available. Using CPU.


In [13]:
def load_model(model_id):
    global pipe
    if pipe is not None:
        del pipe
        torch.cuda.empty_cache()

    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        use_safetensors=True,
        variant="fp16"
    ).to("cpu")  # Change to "cpu"
    return pipe

In [9]:
STYLE_PRESETS = {
    "Modern": "clean lines, geometric shapes, gradient colors",
    "Vintage": "textured, distressed look, muted colors",
    "Tech": "neon glow, circuit patterns, dark background",
    "Organic": "hand-drawn, flowing shapes, natural colors"
}

In [ ]:
!pip install potrace # Install the correct Python bindings for potrace
from PIL import Image
import potrace # Import the potrace module

def raster_to_svg(pil_image):
    bitmap = potrace.Bitmap(pil_image.convert("L")) # Use potrace to create the Bitmap object
    path = bitmap.trace()
    return path

In [ ]:
# Add to generation function
import gc

def clear_memory():
    torch.cuda.empty_cache()
    gc.collect()
    pipe = None  # Force reload for memory-sensitive operations

In [ ]:
%%capture
# 1. Install system requirements first
!sudo apt-get update
!sudo apt-get install -y potrace  # Core C library
!pip install pypotrace            # Python bindings

# 2. Verify installation
!ldd /usr/lib/libpotrace.so  # Check library links
!pip show pypotrace          # Verify Python package

# 3. Restart Colab runtime after this cell
# Runtime -> Restart runtime

In [ ]:
from PIL import Image
import numpy as np

# Test after installation
try:
    import pypotrace
    def vectorize_with_potrace(img):
        bitmap = pypotrace.Bitmap(np.array(img.convert('L')) < 128)
        return bitmap.trace()
except:
    print("Using fallback vectorization")
    def vectorize_with_potrace(img):
        return simple_vectorize(img)

# Usage
# Check if the file exists before opening it
import os
if os.path.exists("logo.png"):
    logo = Image.open("logo.png")
    vector_data = vectorize_with_potrace(logo)
else:
    print("The file 'logo.png' does not exist. Please ensure it has been generated and saved.")

In [ ]:
!pip install scour
import scour

def optimize_svg(svg_str):
    options = scour.sanitizeOptions()
    options.enable_viewboxing = True
    options.strip_comments = True
    options.strip_ids = True
    options.shorten_ids = True
    return scour.scourString(svg_str, options)

!pip install scour
import scour

def optimize_svg(svg_str):
    options = scour.sanitizeOptions()
    options.enable_viewboxing = True
    options.strip_comments = True
    options.strip_ids = True
    options.shorten_ids = True
    return scour.scourString(svg_str, options)

# Assume you have raw SVG data in a string variable called 'my_raw_svg'
# This is just an example, replace with your actual raw SVG data
my_raw_svg = '<svg width="100" height="100"><circle cx="50" cy="50" r="40" stroke="green" stroke-width="4" fill="yellow" /></svg>'

In [ ]:
from xml.dom import minidom

def add_metadata(svg_str, designer="AutoLogoAI"):
    doc = minidom.parseString(svg_str)
    metadata = doc.createElementNS("http://www.w3.org/2000/svg", "metadata")
    metadata.appendChild(doc.createComment("Generated by AutoLogoAI"))
    doc.documentElement.insertBefore(metadata, doc.documentElement.firstChild)
    return doc.toxml()

# Usage
svg_with_meta = add_metadata(optimized_svg)

In [ ]:
!pip install scour
from xml.dom import minidom
import scour


def optimize_svg(svg_str):
    options = scour.sanitizeOptions()
    options.enable_viewboxing = True
    options.strip_comments = True
    options.strip_ids = True
    options.shorten_ids = True
    return scour.scourString(svg_str, options)


def add_metadata(svg_str, designer="AutoLogoAI"):
    doc = minidom.parseString(svg_str)
    metadata = doc.createElementNS("http://www.w3.org/2000/svg", "metadata")
    metadata.appendChild(doc.createComment("Generated by AutoLogoAI"))
    doc.documentElement.insertBefore(metadata, doc.documentElement.firstChild)
    return doc.toxml()

# Assume you have raw SVG data in a string variable called 'my_raw_svg'
# This is just an example, replace with your actual raw SVG data
my_raw_svg = '<svg width="100" height="100"><circle cx="50" cy="50" r="40" stroke="green" stroke-width="4" fill="yellow" /></svg>'

# Call optimize_svg with the raw SVG data to generate optimized_svg
optimized_svg = optimize_svg(my_raw_svg)

# Now you can call add_metadata with optimized_svg
svg_with_meta = add_metadata(optimized_svg)
print(svg_with_meta) # Print the SVG with metadata

In [ ]:
from scour import scour

def optimize_svg(svg_str):
    # Create options object through proper initialization
    options = scour.sanitizeOptions()  # Use scour.sanitizeOptions() to get options

    # Set optimization parameters
    options.enable_viewboxing = True
    options.strip_comments = True
    options.strip_ids = True
    options.shorten_ids = True
    options.remove_metadata = True
    options.indent_type = 'none'

    # Perform optimization
    return scour.scourString(svg_str, options)

# Usage example
# Replace this with your actual raw SVG data
raw_svg_data = '<svg width="100" height="100"><circle cx="50" cy="50" r="40" stroke="green" stroke-width="4" fill="yellow" /></svg>'
optimized_svg = optimize_svg(raw_svg_data)
print(optimized_svg) # Print the optimized SVG

In [ ]:
from xml.dom import minidom

def add_metadata(svg_str, designer="AutoLogoAI"):
    doc = minidom.parseString(svg_str)
    metadata = doc.createElementNS("http://www.w3.org/2000/svg", "metadata")
    metadata.appendChild(doc.createComment("Generated by AutoLogoAI"))
    doc.documentElement.insertBefore(metadata, doc.documentElement.firstChild)
    return doc.toxml()

# Usage
svg_with_meta = add_metadata(optimized_svg)

In [ ]:
!pip install cairosvg
from cairosvg import svg2png, svg2pdf

def export_formats(svg_str, base_name="logo"):
    # PNG
    svg2png(bytestring=svg_str, write_to=f"{base_name}.png")

    # PDF
    svg2pdf(bytestring=svg_str, write_to=f"{base_name}.pdf")

    # EPS (for print)
    !inkscape {base_name}.svg --export-eps={base_name}.eps

    # Save original SVG
    with open(f"{base_name}.svg", "w") as f:
        f.write(svg_str)

In [ ]:
import re

def generate_color_variants(svg_str, palette):
    variants = []
    for name, colors in palette.items():
        variant = svg_str
        for i, color in enumerate(colors):
            variant = re.sub(f'fill="#([0-9a-fA-F]{{6}})"',
                           f'fill="{color}"',
                           variant, count=1)
        variants.append((name, variant))
    return variants

# Usage
palette = {
    "Dark Theme": ["#2A2A2A", "#FFFFFF"],
    "Corporate Blue": ["#1E3A8A", "#BFDBFE"]
}
variants = generate_color_variants(svg_with_meta, palette)

In [ ]:
!pip install svglib
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF

def validate_svg(svg_path):
    try:
        drawing = svg2rlg(svg_path)
        renderPDF.drawToFile(drawing, "temp.pdf")
        return True
    except Exception as e:
        print(f"Invalid SVG: {str(e)}")
        return False

In [ ]:
from google.colab import files
import zipfile

def create_delivery_package():
    with zipfile.ZipFile('logo_package.zip', 'w') as zipf:
        zipf.write('logo.svg')
        zipf.write('logo.png')
        zipf.write('logo.pdf')
        zipf.write('style-guide.pdf')

    files.download('logo_package.zip')

In [ ]:
def create_web_components(svg_str):
    # Web Component
    web_component = f"""
    <script>
    class AutoLogo extends HTMLElement {{
        connectedCallback() {{
            this.innerHTML = `{svg_str}`;
        }}
    }}
    customElements.define('auto-logo', AutoLogo);
    </script>
    """

    # React Component
    react_component = f"""
    import React from 'react';
    export default () => (
        <div dangerouslySetInnerHTML={{ __html: `{svg_str}` }} />
    );
    """

    return {
        'web_component': web_component,
        'react_component': react_component
    }